In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# df = pd.read_csv("/content/sample_data/ThoracicSurgery.csv")
df = pd.read_csv("ThoracicSurgery.csv")
df

,id,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,1,DGN2,2.88,2.16,PRZ1,F,F,F,T,T,OC14,F,F,F,T,F,60,F
1,2,DGN3,3.40,1.88,PRZ0,F,F,F,F,F,OC12,F,F,F,T,F,51,F
2,3,DGN3,2.76,2.08,PRZ1,F,F,F,T,F,OC11,F,F,F,T,F,59,F
3,4,DGN3,3.68,3.04,PRZ0,F,F,F,F,F,OC11,F,F,F,F,F,54,F
4,5,DGN3,2.44,0.96,PRZ2,F,T,F,T,T,OC11,F,F,F,T,F,73,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,466,DGN2,3.88,2.12,PRZ1,F,F,F,T,F,OC13,F,F,F,T,F,63,F
466,467,DGN3,3.76,3.12,PRZ0,F,F,F,F,F,OC11,F,F,F,T,F,61,F
467,468,DGN3,3.04,2.08,PRZ1,F,F,F,T,F,OC13,F,F,F,F,F,52,F
468,469,DGN3,1.96,1.68,PRZ1,F,F,F,T,T,OC12,F,F,F,T,F,79,F


In [3]:
df = df.drop(columns = ["id", "DGN", "PRE6", "PRE7", 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19',
             'PRE25', 'PRE30', 'PRE32'])

In [4]:
cleaned_df = df

In [5]:
cleaned_df['PRE14'] = np.where(cleaned_df['PRE14'] == 'OC11', '11', cleaned_df['PRE14'] )
cleaned_df['PRE14'] = np.where(cleaned_df['PRE14'] == 'OC12', '12', cleaned_df['PRE14'] )
cleaned_df['PRE14'] = np.where(cleaned_df['PRE14'] == 'OC13', '13', cleaned_df['PRE14'] )
cleaned_df['PRE14'] = np.where(cleaned_df['PRE14'] == 'OC14', '14', cleaned_df['PRE14'] )
cleaned_df['Risk1Yr'] = np.where(cleaned_df['Risk1Yr'] == 'T', '0', cleaned_df['Risk1Yr'] )
cleaned_df['Risk1Yr'] = np.where(cleaned_df['Risk1Yr'] == 'F', '1', cleaned_df['Risk1Yr'] )
cleaned_df.head()

,PRE4,PRE5,PRE14,AGE,Risk1Yr
0,2.88,2.16,14,60,1
1,3.40,1.88,12,51,1
2,2.76,2.08,11,59,1
3,3.68,3.04,11,54,1
4,2.44,0.96,11,73,0


In [6]:
cleaned_df['PRE14'] = cleaned_df['PRE14'].astype(int)

In [11]:
survived_df = cleaned_df[cleaned_df["Risk1Yr"] == '1']
survived_df.rename(columns={"PRE4": "FVC", "PRE5": "FEV",
                  "PRE14": "TumorSize", "AGE": "Age"}, inplace=True)
survived_df = survived_df.drop("Risk1Yr", axis=1)
survived_df.head()

C:\Users\petej\Anaconda3\envs\PythonAdv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,FVC,FEV,TumorSize,Age
0,2.88,2.16,14,60
1,3.40,1.88,12,51
2,2.76,2.08,11,59
3,3.68,3.04,11,54
5,2.48,1.88,11,51


In [8]:
survived_df.describe(include='all').loc[['mean']]

,FVC,FEV,TumorSize,Age
mean,3.298525,4.779375,11.685,62.3925


In [9]:
notSurvived_df = cleaned_df[cleaned_df["Risk1Yr"] == '0']
notSurvived_df.rename(columns={"PRE4": "FVC", "PRE5": "FEV",
                  "PRE14": "TumorSize", "AGE": "Age"}, inplace=True)
notSurvived_df = notSurvived_df.drop("Risk1Yr", axis=1)
notSurvived_df.head()

C:\Users\petej\Anaconda3\envs\PythonAdv\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,FVC,FEV,TumorSize,Age
4,2.44,0.96,11,73
6,4.36,3.28,12,59
7,3.19,2.50,11,66
13,3.98,3.06,14,80
24,4.32,3.20,11,58


In [10]:
notSurvived_df.describe(include='all').loc[['mean']]

,FVC,FEV,TumorSize,Age
mean,3.185143,3.364857,12.028571,63.342857
